In [0]:
# TruffleHog Secret Scanner for Databricks Notebooks

## Overview
This notebook scans Databricks workspace notebooks for exposed secrets using TruffleHog. It integrates with the Security Analysis Tool (SAT) to provide comprehensive secret detection across your Databricks environment.

## Features
- Scans all notebooks modified within a specified timeframe
- Uses custom detectors for Databricks-specific tokens
- Excludes built-in Databricks tokens to reduce false positives  
- Provides detailed reporting with SHA-256 hashed secrets for security
- Handles pagination for large workspaces
- Includes proper error handling and rate limiting

## Prerequisites
- Databricks workspace with appropriate permissions
- Access to install packages and run shell commands
- Valid Databricks API token (automatically extracted from notebook context)

---

## Step 1: Install Dependencies and Setup TruffleHog

This cell installs required Python packages and downloads TruffleHog binary. It also creates the custom detector configuration.


In [ ]:
%sh 
# Install required Python packages
pip install requests pyyaml

# Download and install TruffleHog binary to /tmp directory
echo "Installing TruffleHog..."
curl -sSfL https://raw.githubusercontent.com/trufflesecurity/trufflehog/main/scripts/install.sh | sh -s -- -b /tmp

echo "Setup completed successfully!"
echo "TruffleHog binary location: /tmp/trufflehog"
echo "Configuration will be loaded from: /Workspace/Repos/.../configs/trufflehog_detectors.yaml"


## Step 2: Configuration and Authentication

This cell sets up configuration constants and extracts Databricks authentication context.


In [0]:
# Import required libraries
import os
import requests
import time
import json
import base64
import subprocess
import hashlib
import logging
import yaml
from datetime import timedelta, datetime
from urllib.parse import quote
from typing import Dict, List, Optional, Any

# Configure logging for better debugging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def load_config_from_file():
    """Load configuration from the external YAML file in the configs directory."""
    try:
        # Get the current notebook's directory and construct path to config file
        notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        repo_root = "/".join(notebook_path.split("/")[:-3])  # Go up from notebooks/Includes/scan_secrets
        config_path = f"{repo_root}/configs/trufflehog_detectors.yaml"
        
        logger.info(f"Loading configuration from: {config_path}")
        
        # Read the configuration file
        with open(config_path, 'r') as file:
            config_data = yaml.safe_load(file)
        
        return config_data
    except Exception as e:
        logger.warning(f"Could not load external config file: {str(e)}")
        logger.info("Using default configuration")
        # Fallback to default configuration
        return {
            "detectors": [
                {"name": "DkeaToken", "keywords": ["dkea"], "regex": {"id": "(?i)\\b(dkea[a-h0-9]{32})"}},
                {"name": "DapiToken", "keywords": ["dapi"], "regex": {"id": "(?i)\\b(dapi[a-h0-9]{32})"}},
                {"name": "DoseToken", "keywords": ["dose"], "regex": {"id": "(?i)\\b(dose[a-h0-9]{32})"}}
            ],
            "settings": {
                "excluded_detectors": ["DatabricksToken"],
                "rate_limiting": {"api_sleep_seconds": 10},
                "search_settings": {"page_size": 50, "days_back": 1}
            }
        }

def create_trufflehog_config(config_data: Dict[str, Any]) -> str:
    """Create TruffleHog configuration file from loaded config data."""
    config_file_path = "/tmp/trufflehog_config.yaml"
    
    # Extract just the detectors section for TruffleHog
    trufflehog_config = {"detectors": config_data.get("detectors", [])}
    
    try:
        with open(config_file_path, 'w') as file:
            yaml.dump(trufflehog_config, file, default_flow_style=False)
        
        logger.info(f"TruffleHog configuration created at: {config_file_path}")
        return config_file_path
    except Exception as e:
        logger.error(f"Failed to create TruffleHog config file: {str(e)}")
        raise

# Load configuration from external file
config_data = load_config_from_file()

# Configuration class using loaded values
class Config:
    """Configuration class for TruffleHog scanning parameters"""
    
    # File paths
    TRUFFLEHOG_BINARY = "/tmp/trufflehog"
    TRUFFLEHOG_CONFIG = create_trufflehog_config(config_data)
    TEMP_NOTEBOOKS_DIR = config_data.get("settings", {}).get("file_paths", {}).get("temp_notebooks", "/tmp/notebooks")
    RESULTS_LOG_FILE = config_data.get("settings", {}).get("file_paths", {}).get("results_log", "/tmp/trufflehog_scan_results.json")
    
    # API settings from config
    API_SLEEP_SECONDS = config_data.get("settings", {}).get("rate_limiting", {}).get("api_sleep_seconds", 10)
    PAGE_SIZE = config_data.get("settings", {}).get("search_settings", {}).get("page_size", 50)
    DAYS_BACK = config_data.get("settings", {}).get("search_settings", {}).get("days_back", 1)
    
    # TruffleHog settings from config
    EXCLUDED_DETECTORS = config_data.get("settings", {}).get("excluded_detectors", ["DatabricksToken"])
    
# Extract Databricks authentication context
# These are automatically available in Databricks notebooks
try:
    token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().getOrElse(None)
    base_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().getOrElse(None)
    
    if not token or not base_url:
        raise ValueError("Unable to extract Databricks authentication context")
        
    logger.info(f"Successfully extracted Databricks context. Base URL: {base_url}")
    
except Exception as e:
    logger.error(f"Failed to extract Databricks context: {str(e)}")
    raise

# Create temporary directories if they don't exist
os.makedirs(Config.TEMP_NOTEBOOKS_DIR, exist_ok=True)
logger.info(f"Temporary directory created: {Config.TEMP_NOTEBOOKS_DIR}")

print("✅ Configuration loaded from external file and authentication setup completed successfully!")
print(f"📁 Config file: {Config.TRUFFLEHOG_CONFIG}")
print(f"🔧 Loaded {len(config_data.get('detectors', []))} custom detectors")
print(f"⚙️  API sleep: {Config.API_SLEEP_SECONDS}s, Page size: {Config.PAGE_SIZE}, Days back: {Config.DAYS_BACK}")

## Step 3: Utility Functions

This cell defines all the utility functions for API interactions, file operations, and secret scanning.


In [ ]:
# Utility Functions for TruffleHog Secret Scanning

def get_yesterday_utc_midnight() -> int:
    """
    Get yesterday's date in UTC at midnight as milliseconds timestamp.
    
    Returns:
        int: Timestamp in milliseconds for yesterday at 00:00 UTC
    """
    today_utc_midnight = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
    yesterday_utc_midnight = today_utc_midnight - timedelta(days=Config.DAYS_BACK)
    return int(yesterday_utc_midnight.timestamp() * 1000)

def convert_time_to_databricks_format(env_time: int) -> int:
    """
    Convert environment time to Databricks format.
    
    Args:
        env_time (int): Time in milliseconds
        
    Returns:
        int: Time in Databricks format (milliseconds)
    """
    return int(env_time)

def generate_sha256_hash(secret: str) -> str:
    """
    Generate SHA-256 hash of a secret for secure logging.
    
    Args:
        secret (str): The secret string to hash
        
    Returns:
        str: SHA-256 hash in hexadecimal format
    """
    secret_bytes = secret.encode("utf-8")
    sha = hashlib.sha256()
    sha.update(secret_bytes)
    return sha.hexdigest()

def make_api_request(url: str, headers: Dict[str, str], data: Optional[Dict[str, Any]] = None) -> Optional[Dict[str, Any]]:
    """
    Make API request to Databricks with proper error handling.
    
    Args:
        url (str): API endpoint URL
        headers (Dict[str, str]): Request headers
        data (Optional[Dict[str, Any]]): Request payload
        
    Returns:
        Optional[Dict[str, Any]]: JSON response or None if error
    """
    try:
        response = requests.get(url, headers=headers, json=data, timeout=30)
        
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            logger.warning(f"Rate limit hit for URL: {url}. Waiting before retry...")
            time.sleep(Config.API_SLEEP_SECONDS * 2)  # Wait longer for rate limits
            return None
        else:
            logger.warning(f"API request failed. URL: {url}, Status: {response.status_code}")
            return None
            
    except requests.exceptions.Timeout:
        logger.error(f"Request timeout for URL: {url}")
        return None
    except requests.exceptions.RequestException as e:
        logger.error(f"Request error for URL: {url}. Error: {str(e)}")
        return None

def check_notebook_status(notebook_path: str) -> int:
    """
    Check if a notebook exists and is accessible.
    
    Args:
        notebook_path (str): Path to the notebook
        
    Returns:
        int: HTTP status code (200=accessible, 403=no permission, 404=not found)
    """
    check_url = f"{base_url}/api/2.0/workspace/get-status?path={notebook_path}"
    headers = {"Authorization": f"Bearer {token}"}

    try:
        response = requests.get(check_url, headers=headers, timeout=30)
        return response.status_code
    except requests.exceptions.RequestException as e:
        logger.error(f"Error checking notebook status for {notebook_path}: {str(e)}")
        return 500  # Internal server error

def export_notebook_content(notebook_path: str) -> Optional[Dict[str, Any]]:
    """
    Export notebook content from Databricks workspace.
    
    Args:
        notebook_path (str): Path to the notebook
        
    Returns:
        Optional[Dict[str, Any]]: Notebook export response or None if error
    """
    try:
        headers = {"Authorization": f"Bearer {token}"}
        url = f"{base_url}/api/2.0/workspace/export?path={notebook_path}"
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logger.error(f"Error exporting notebook content for {notebook_path}: {str(e)}")
        return None

def decode_and_write_content(content: str, output_path: str) -> bool:
    """
    Decode base64 content and write to file.
    
    Args:
        content (str): Base64 encoded content
        output_path (str): Path to write decoded content
        
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        decoded_content = base64.b64decode(content).decode("utf-8")
        with open(output_path, "w", encoding="utf-8") as file:
            file.write(decoded_content)
        return True
    except Exception as e:
        logger.error(f"Error writing content to {output_path}: {str(e)}")
        return False

def scan_for_secrets(file_path: str) -> Optional[str]:
    """
    Run TruffleHog scan on a file to detect secrets.
    
    Args:
        file_path (str): Path to file to scan
        
    Returns:
        Optional[str]: TruffleHog output in JSON format or None if error
    """
    # Build TruffleHog command with proper configuration
    excluded_detectors = ",".join(Config.EXCLUDED_DETECTORS)
    trufflehog_command = (
        f"{Config.TRUFFLEHOG_BINARY} filesystem {file_path} "
        f"--exclude-detectors {excluded_detectors} "
        f"--no-update --config {Config.TRUFFLEHOG_CONFIG} -j"
    )
    
    try:
        result = subprocess.run(
            trufflehog_command, 
            shell=True, 
            check=True, 
            capture_output=True, 
            text=True,
            timeout=300  # 5 minute timeout
        )
        return result.stdout
    except subprocess.TimeoutExpired:
        logger.error(f"TruffleHog scan timed out for file: {file_path}")
        return None
    except subprocess.CalledProcessError as e:
        logger.error(f"TruffleHog scan failed for file: {file_path}. Error: {e}")
        if e.stderr:
            logger.error(f"TruffleHog stderr: {e.stderr}")
        return None

def process_trufflehog_output(trufflehog_output: str) -> List[Dict[str, str]]:
    """
    Process TruffleHog JSON output and extract relevant information.
    
    Args:
        trufflehog_output (str): Raw TruffleHog output in JSON format
        
    Returns:
        List[Dict[str, str]]: List of detected secrets with hashed values
    """
    results = []
    
    if not trufflehog_output or not trufflehog_output.strip():
        return results
    
    for line in trufflehog_output.strip().splitlines():
        try:
            data = json.loads(line)
            detector_name = data.get("DetectorName", "Unknown")
            raw_value = data.get("Raw", "")
            
            if raw_value:
                # Generate SHA-256 hash for security (don't log actual secrets)
                raw_sha = generate_sha256_hash(raw_value)
                
                # Add metadata about the detection
                result = {
                    "DetectorName": detector_name,
                    "Raw_SHA": raw_sha,
                    "SourceFile": data.get("SourceMetadata", {}).get("Data", {}).get("Filesystem", {}).get("file", "Unknown"),
                    "Verified": data.get("Verified", False)
                }
                results.append(result)
                
        except json.JSONDecodeError as e:
            logger.warning(f"Failed to parse TruffleHog output line: {line}. Error: {str(e)}")
            continue
    
    return results

print("✅ Utility functions defined successfully!")


## Step 4: Main Scanning Functions

This cell contains the main functions for processing notebooks and orchestrating the secret scanning workflow.


In [ ]:
# Main Scanning Functions

def scan_notebook_for_secrets(notebook_path: str, object_id: str) -> Optional[List[Dict[str, str]]]:
    """
    Scan a single notebook for secrets using TruffleHog.
    
    Args:
        notebook_path (str): Path to the notebook in Databricks workspace
        object_id (str): Unique identifier for the notebook
        
    Returns:
        Optional[List[Dict[str, str]]]: List of detected secrets or None if error
    """
    try:
        # Check if notebook exists and is accessible
        notebook_status = check_notebook_status(notebook_path)
        
        if notebook_status == 200:
            # Export notebook content
            export_response = export_notebook_content(notebook_path)
            if export_response is None:
                logger.warning(f"Failed to export notebook content: {notebook_path}")
                return None
                
            content = export_response.get("content")
            if not content:
                logger.warning(f"No content found in notebook: {notebook_path}")
                return None
            
            # Write content to temporary file
            output_file_path = os.path.join(Config.TEMP_NOTEBOOKS_DIR, f"notebook_content_{object_id}.txt")
            if not decode_and_write_content(content, output_file_path):
                logger.error(f"Failed to write notebook content to file: {output_file_path}")
                return None
                
            logger.info(f"Notebook content exported to: {output_file_path}")
            
            # Scan for secrets using TruffleHog
            trufflehog_output = scan_for_secrets(output_file_path)
            if trufflehog_output is None:
                logger.warning(f"TruffleHog scan failed for: {notebook_path}")
                return None
            
            # Process TruffleHog output
            results = process_trufflehog_output(trufflehog_output)
            
            if results:
                logger.info(f"Found {len(results)} potential secrets in notebook: {notebook_path}")
                # Log results securely (with hashed secrets)
                for result in results:
                    logger.info(f"Secret detected - Type: {result['DetectorName']}, SHA: {result['Raw_SHA'][:16]}...")
            else:
                logger.info(f"No secrets found in notebook: {notebook_path}")
            
            # Clean up temporary file
            try:
                os.remove(output_file_path)
            except OSError as e:
                logger.warning(f"Failed to clean up temporary file {output_file_path}: {str(e)}")
            
            return results
            
        elif notebook_status == 403:
            logger.warning(f"Access denied for notebook: {notebook_path}")
            return None
        elif notebook_status == 404:
            logger.warning(f"Notebook not found: {notebook_path}")
            return None
        else:
            logger.warning(f"Unexpected status {notebook_status} for notebook: {notebook_path}")
            return None
            
    except Exception as e:
        logger.error(f"Error scanning notebook {notebook_path}: {str(e)}")
        return None

def process_search_response(response: Dict[str, Any], results_list: List[Dict[str, Any]], 
                          output_filename: Optional[str] = None) -> Optional[str]:
    """
    Process search API response and scan notebooks for secrets.
    
    Args:
        response (Dict[str, Any]): API response from unified search
        results_list (List[Dict[str, Any]]): List to store notebook metadata
        output_filename (Optional[str]): File to log notebook metadata
        
    Returns:
        Optional[str]: Next page token for pagination or None if no more pages
    """
    if not response:
        logger.warning("Empty response received")
        return None
        
    results = response.get("results", [])
    logger.info(f"Processing {len(results)} notebooks from search response")
    
    for notebook in results:
        notebook_id = notebook.get("id", "")
        notebook_name = notebook.get("name", "")
        parent_path = notebook.get("workspace_path", "")
        
        if not notebook_id or not notebook_name:
            logger.warning("Skipping notebook with missing ID or name")
            continue
            
        # Construct full notebook path
        temp_path = f"{parent_path}/{notebook_name}"
        path = quote(temp_path)
        
        logger.info(f"Processing notebook: {notebook_id} - {temp_path}")
        
        # Store notebook metadata
        notebook_metadata = {"object_id": notebook_id, "path": path, "name": notebook_name}
        results_list.append(notebook_metadata)
        
        # Log to file if specified
        if output_filename:
            try:
                with open(output_filename, mode="a", encoding="utf-8") as output_file:
                    json.dump(notebook_metadata, output_file)
                    output_file.write("\n")
            except IOError as e:
                logger.error(f"Failed to write to output file {output_filename}: {str(e)}")
        
        # Scan notebook for secrets
        secret_results = scan_notebook_for_secrets(path, notebook_id)
        
        if secret_results:
            # Store results with notebook metadata
            notebook_metadata["secrets_found"] = len(secret_results)
            notebook_metadata["secret_details"] = secret_results
            
            # Log summary
            print(f"🚨 SECRETS DETECTED in {temp_path}:")
            print(json.dumps(secret_results, indent=2))
        else:
            notebook_metadata["secrets_found"] = 0
            print(f"✅ No secrets found in {temp_path}")
    
    # Return next page token for pagination
    return response.get("next_page_token")

print("✅ Main scanning functions defined successfully!")


## Step 5: Execute Secret Scanning

This cell executes the main scanning workflow to search for notebooks and scan them for secrets.


In [ ]:
# Execute TruffleHog Secret Scanning Workflow

def main_scanning_workflow():
    """
    Main function to orchestrate the secret scanning process.
    """
    print("🔍 Starting TruffleHog Secret Scanning Workflow")
    print("=" * 60)
    
    # Get time range for notebook search
    # Use environment variable TIME if provided, otherwise use yesterday's midnight
    env_time = os.environ.get("TIME")
    if env_time:
        try:
            last_edited_after = convert_time_to_databricks_format(int(env_time))
            logger.info(f"Using provided TIME environment variable: {env_time}")
        except ValueError:
            logger.warning(f"Invalid TIME environment variable: {env_time}. Using default.")
            last_edited_after = get_yesterday_utc_midnight()
    else:
        last_edited_after = get_yesterday_utc_midnight()
        logger.info(f"Using default time range: last {Config.DAYS_BACK} day(s)")
    
    # Initialize tracking variables
    results_list = []
    total_notebooks_processed = 0
    total_secrets_found = 0
    notebooks_with_secrets = 0
    
    # Setup API request parameters
    url = f"{base_url}/api/2.0/search-midtier/unified-search"
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    data = {
        "query": {"query": ""},
        "filters": {"result_types": ["NOTEBOOK"], "last_edited_after": last_edited_after},
        "page_size": Config.PAGE_SIZE,
    }
    
    print(f"📅 Searching for notebooks modified after: {datetime.fromtimestamp(last_edited_after/1000).strftime('%Y-%m-%d %H:%M:%S UTC')}")
    print(f"📄 Page size: {Config.PAGE_SIZE}")
    print()
    
    # Pagination loop
    next_page_token = ""  # Start with empty token
    page_number = 1
    
    try:
        while next_page_token is not None:
            print(f"📖 Processing page {page_number}...")
            
            # Add pagination token to request
            if next_page_token:
                data["page_token"] = next_page_token
            elif "page_token" in data:
                del data["page_token"]  # Remove token for first request
            
            # Make API request
            response = make_api_request(url, headers, data)
            
            if response is None:
                logger.warning(f"Failed to get response for page {page_number}")
                break
            
            # Process response and scan notebooks
            page_start_time = time.time()
            next_page_token = process_search_response(response, results_list, Config.RESULTS_LOG_FILE)
            page_end_time = time.time()
            
            # Update counters
            page_notebooks = len(response.get("results", []))
            total_notebooks_processed += page_notebooks
            
            # Count secrets found in this page
            page_secrets = sum(1 for notebook in results_list[-page_notebooks:] if notebook.get("secrets_found", 0) > 0)
            notebooks_with_secrets += page_secrets
            total_secrets_found += sum(notebook.get("secrets_found", 0) for notebook in results_list[-page_notebooks:])
            
            print(f"   ⏱️  Page {page_number} completed in {page_end_time - page_start_time:.2f} seconds")
            print(f"   📊 Notebooks processed: {page_notebooks}, Secrets found: {sum(notebook.get('secrets_found', 0) for notebook in results_list[-page_notebooks:])}")
            print()
            
            page_number += 1
            
            # Rate limiting - sleep between API calls
            if next_page_token is not None:
                logger.info(f"Sleeping for {Config.API_SLEEP_SECONDS} seconds to prevent rate limiting...")
                time.sleep(Config.API_SLEEP_SECONDS)
        
        # Final summary
        print("🎉 Secret Scanning Completed!")
        print("=" * 60)
        print(f"📊 Total notebooks processed: {total_notebooks_processed}")
        print(f"🔍 Notebooks with secrets: {notebooks_with_secrets}")
        print(f"🚨 Total secrets detected: {total_secrets_found}")
        
        if notebooks_with_secrets > 0:
            print(f"⚠️  Security Alert: {notebooks_with_secrets} notebook(s) contain potential secrets!")
            print("   Please review the detailed results above and take appropriate action.")
        else:
            print("✅ No secrets detected in any notebooks. Great job!")
        
        print(f"📝 Detailed results logged to: {Config.RESULTS_LOG_FILE}")
        
        # Return summary statistics
        return {
            "total_notebooks": total_notebooks_processed,
            "notebooks_with_secrets": notebooks_with_secrets,
            "total_secrets": total_secrets_found,
            "results": results_list
        }
        
    except KeyboardInterrupt:
        print("\n⏹️  Scanning interrupted by user")
        logger.info("Scanning workflow interrupted by user")
        return None
    except Exception as e:
        logger.error(f"Unexpected error in scanning workflow: {str(e)}")
        print(f"❌ Error occurred during scanning: {str(e)}")
        return None

# Execute the scanning workflow
if __name__ == "__main__":
    scan_results = main_scanning_workflow()


In [0]:
# Results Analysis and Cleanup

# Display scan results summary if available
if 'scan_results' in locals() and scan_results:
    print("📈 Detailed Scan Results Analysis")
    print("=" * 50)
    
    # Group results by secret type
    secret_types = {}
    for notebook in scan_results.get("results", []):
        if notebook.get("secrets_found", 0) > 0:
            for secret in notebook.get("secret_details", []):
                detector_name = secret.get("DetectorName", "Unknown")
                if detector_name not in secret_types:
                    secret_types[detector_name] = 0
                secret_types[detector_name] += 1
    
    if secret_types:
        print("🔍 Secret Types Detected:")
        for secret_type, count in sorted(secret_types.items()):
            print(f"   • {secret_type}: {count} occurrence(s)")
        print()
    
    # Show notebooks with most secrets
    notebooks_by_secrets = sorted(
        [nb for nb in scan_results.get("results", []) if nb.get("secrets_found", 0) > 0],
        key=lambda x: x.get("secrets_found", 0),
        reverse=True
    )
    
    if notebooks_by_secrets:
        print("📚 Top Notebooks with Secrets:")
        for i, notebook in enumerate(notebooks_by_secrets[:5]):  # Show top 5
            print(f"   {i+1}. {notebook.get('name', 'Unknown')} - {notebook.get('secrets_found', 0)} secret(s)")
        print()

# Cleanup and file operations
print("🧹 Cleanup Operations")
print("=" * 30)

# List temporary files created
print("📁 Temporary files in /tmp/notebooks:")
%sh ls -la /tmp/notebooks/ 2>/dev/null || echo "Directory not found or empty"

print("\n📄 Configuration files:")
%sh ls -la /tmp/trufflehog* 2>/dev/null || echo "No TruffleHog config files found"

print("\n📊 Results log file:")
%sh ls -la /tmp/trufflehog_scan_results.json 2>/dev/null || echo "No results log file found"

print("\n✅ Cleanup completed. Temporary files will be automatically removed when the cluster terminates.")

# Display final recommendations
print("\n🎯 Next Steps and Recommendations")
print("=" * 40)
print("1. 🔍 Review any detected secrets immediately")
print("2. 🔄 Rotate any exposed credentials")
print("3. 📝 Update notebooks to remove hardcoded secrets")
print("4. 🔐 Use Databricks secrets or environment variables instead")
print("5. 📅 Schedule regular secret scans as part of your security workflow")
print("6. 📋 Consider integrating this scan into your CI/CD pipeline")

if 'scan_results' in locals() and scan_results and scan_results.get("notebooks_with_secrets", 0) > 0:
    print("\n⚠️  IMMEDIATE ACTION REQUIRED:")
    print("   Secrets were detected in your notebooks. Please address them promptly!")
else:
    print("\n✅ No immediate action required - no secrets detected.")